In [59]:
import os
import pandas as pd

In [48]:
instance_set = "miplib_5000"

In [49]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "gurobipy.gurobierror: out of memory": [],
    "could not be solved in 14400 seconds. skipping.": [],
    "gurobipy.gurobierror: unable to create presolved model": [],
    "objective could not be perturbed": [],
    "invalid value encountered in double_scalars": [],
}

# runs that weren't run
no_go = []

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

for i, file_name in enumerate(os.listdir(os.path.join("instances", instance_set))):
    
    # skip anything not a mip
    if not file_name.endswith(".mps"):
        continue
        
    # get the model name
    stem = file_name[:-4]
    
    # get the error file path
    err_pth = os.path.join("outfiles", stem + ".err")
    
    # check if the series wasn't run
    if not os.path.exists(err_pth):
        no_go.append(stem)
    
    # check if the series ran with no errors or warnings
    elif os.path.getsize(err_pth) == 0:
        empty.append(stem)
        
    else:
        # read in file
        with open(err_pth, "r") as f:
            text = f.read().lower()
            
        # check for error codes
        found_code = False
        for code in err:
            if code in text:
                err[code].append(file_name)
                found_code = True
                break
        if not found_code:
            other.append(file_name)

In [50]:
no_go

[]

In [51]:
len(empty) / (i + 1)

0.7482837528604119

In [65]:
print(err["gurobipy.gurobierror: out of memory"])
print(len(err["gurobipy.gurobierror: out of memory"]) / (i + 1))
df = pd.read_csv("more_memory.csv", index_col=0)
for file_name in err["gurobipy.gurobierror: out of memory"]:
    df.loc[file_name, "memory"] = 16
df["memory"].astype(int)
df.to_csv("more_memory.csv")

['pigeon-16.mps', 'ns1606230.mps', 'neos-3046601-motu.mps', 'neos-1121679.mps', 'neos-5078479-escaut.mps', 'm100n500k4r1.mps', 'neos-3592146-hawea.mps', 'neos-3024952-loue.mps', 'mc11.mps', 'bppc6-02.mps', 'neos-1464762.mps', 'neos-3762025-ognon.mps', 'neos-584146.mps', 'ger50-17-ptp-pop-6t.mps', 'neos-3046615-murg.mps', 'neos-1460641.mps', 'neos-1346382.mps', 'supportcase20.mps', 'breastcancer-regularized.mps', 'ta1-UUM.mps', 'neos-1460543.mps', 'neos-1467371.mps', 'neos-582605.mps', 'd20200.mps', 'pigeon-20.mps', 'neos-3373491-avoca.mps', 'probportfolio.mps', 'neos-1429185.mps', 'janos-us-DDM.mps']
0.06636155606407322


In [53]:
len(err["could not be solved in 14400 seconds. skipping."]) / (i + 1)

0.10755148741418764

In [54]:
print(err["gurobipy.gurobierror: unable to create presolved model"])
len(err["gurobipy.gurobierror: unable to create presolved model"]) / (i + 1)

[]


0.0

In [55]:
len("objective could not be perturbed") / (i + 1)

0.07322654462242563

In [56]:
len("invalid value encountered in double_scalars") / (i + 1)

0.09839816933638444

In [57]:
print(other)
len(other) /( i + 1)

[]


0.0